In [ ]:
import re
from os import listdir, getcwd
from collections import defaultdict
from itertools import product

import pandas as pd
import numpy as np

from figmenta import output_notebook, autovis, show

pd.options.display.max_rows = 1000

output_notebook()

## Static Experiment PIlot

A brief preview of the raw data:

In [ ]:
raw_df = pd.read_csv('static-pilot-latest.csv')
raw_df.head()

Let's take a look how how often the answers of different algorithms correspond. For a static labeling, we wouldn't expect too many targets to lie near a border.

In [ ]:
# define the parameters we care about
row_params = ['num_episodes','num_labels','random_seed', 'target_color']
col_params = ['algorithm']
val_params = ['answer']

# copy and extract the relevant columns
answer_df = raw_df.copy()[row_params + col_params + val_params]

# reshaping magic
# create an index using all the relelvant parameters
# then unstack it to turn the column parameters in to columns
answer_df = answer_df.set_index(row_params + col_params).unstack(level=col_params)

# flatten and rename the column indices
answer_df.columns = ['_'.join([col, fn]) for fn, col in product(*answer_df.columns.levels)]
answer_df = answer_df.reset_index()

# add a column to compare the answers from the two algorithms
answer_df['same'] = (answer_df['brute-force_answer'] == answer_df['exact-heuristic_answer'])

# count the differnt values in that column
answer_df['same'].value_counts()

Next, a look at the episodes searched and the runtime.

In [ ]:
# define the parameters we care about
row_params = ['num_episodes','num_labels']
val_params = ['total_episodes', 'runtime']
aggfuncs = [np.mean, np.std]

# copy and extract the relevant columns
raw_speed_df = raw_df.copy()[row_params + col_params + val_params]

speed_df = raw_speed_df.pivot_table(
        index=row_params + ['algorithm'],
        values=val_params,
        aggfunc=aggfuncs,
    )

# rename and reorder the column indices
col_levels = speed_df.columns.levels
speed_df.columns = ['_'.join([col, fn]) for fn, col in product(*speed_df.columns.levels)]
col_order = ['_'.join([col, fn]) for col, fn in product(*reversed(col_levels))]
speed_df = speed_df[col_order]
speed_df = speed_df.unstack('algorithm')

speed_df

In [ ]:
plot_df = speed_df.loc[1000].stack().reset_index()
fig = autovis(df=plot_df, xs=['algorithm', 'num_labels'], ys=['runtime_mean'])
show(fig)